In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
# !pip install -q pyspark
# !pip install -U -q PyDrive
# !apt install openjdk-8-jdk-headless -qq
# !pip install -q graphframes
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
!pip install graphframes
from graphframes import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 100 not upgraded.
--2022-01-03 16:23:23--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.224.7.49, 13.224.7.124, 13.224.7.16, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.224.7.49|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [2]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()


In [3]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists
Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


Processing Wiki Dumps


In [4]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)


In [5]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
total_number_of_docs_N = parquetFile.count()
doc_text_pairs = parquetFile.select("text", "id").rdd # Remark text is not enought. what about anchor and title?
# Remark! we can get the amount of documents in the corpus here. (N) 
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

In [6]:
def our_sum(lst):
  summ = 0
  for i in lst:
    summ+=i
  return summ

import json
 
# Opening JSON file
test_set_dict = None
with open('queries_train.json') as json_file:
    test_set_dict = json.load(json_file)
test_set_terms = set(' '.join(' '.join(list(test_set_dict.keys())).split('?')).split())

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
### Remark! understand tokenizing? change it?

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  clear_tokens = [token for token in tokens if token not in all_stopwords]
  ### Remark! Stem clear_tokens? ### 
  set_of_toks = set(clear_tokens)

  len_before = len(set_of_toks)
  len_after = len(set_of_toks - test_set_terms)
  if len_after == len_before:
    return []
  dic = {}
  for clear_tok in clear_tokens:
    if clear_tok in dic:
      dic[clear_tok] += 1
    else:
      dic[clear_tok] = 1
  return [(k, (id, v)) for k,v in dic.items()]  
  
def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key= lambda t : t[0])

def vector_size(docid_to_freq,df_dict): #(12, [('anarchism', 125), ('political', 26), ('philosophy', 6), ('movement', 23), ('sceptical', 1), ('authority', 18), ('rejects', 1), ('involuntary', 1), ('coercive', 2), ('forms', 9), ('hierarchy', 4), ('calls', 3), ('abolition', 3), ('state', 50), ('holds', 2), ('undesirable', 1), ('unnecessary', 1), ('harmful', 1), ('historically', 3), ('far-left', 2), ('usually', 4), ('described', 2), ('alongside', 5), ('libertarian', 14), ('marxism', 6), ('wing', 3), ('socialism', 15), ('socialist', 14), ('strong', 2), ('historical', 4), ('association', 5), ('anti-capitalism', 3), ('history', 5), ('anarchy', 9), ('goes', 1), ('back', 2), ('prehistory', 1), ('humans', 4), ('arguably', 1), ('lived', 1), ('anarchic', 2), ('societies', 2), ('long', 2), ('establishment', 3), ('formal', 3), ('states', 8), ('realms', 1), ('empires', 1), ('rise', 1), ('organised', 2), ('hierarchical', 3), ('bodies', 1), ('scepticism', 1), ('toward', 3), ('rose', 1), ('19th', 6), ('century', 20), ('self-conscious', 1), ('emerged', 3), ('latter', 2), ('half', 2), ('first', 13), ('decades', 3), ('20th', 8), ('anarchist', 89), ('flourished', 2), ('parts', 1), ('world', 13), ('significant', 8), ('role', 6), ('workers', 11), ('struggles', 1), ('emancipation', 1), ('various', 15), ('schools', 13), ('thought', 12), ('formed', 5), ('period', 3), ('anarchists', 70), ('taken', 4), ('part', 8), ('several', 3), ('revolutions', 2), ('notably', 2), ('spanish', 4), ('civil', 5), ('war', 11), ('whose', 6), ('end', 5), ('marked', 1), ('classical', 14), ('era', 12), ('last', 2), ('21st', 3), ('resurgent', 1), ('employs', 1), ('diversity', 2), ('tactics', 15), ('order', 3), ('meet', 1), ('ideal', 8), ('ends', 4), ('broadly', 1), ('separated', 1), ('revolutionary', 10), ('evolutionary', 5), ('overlap', 2), ('two', 7), ('merely', 2), ('descriptive', 1), ('aim', 4), ('bring', 2), ('violent', 6), ('turn', 5), ('past', 2), ('prefigure', 1), ('society', 23), ('would', 14), ('like', 1), ('criticism', 10), ('praxis', 2), ('played', 1), ('diverse', 4), ('areas', 2), ('human', 9), ('criticisms', 1), ('include', 6), ('claims', 4), ('internally', 1), ('inconsistent', 1), ('utopian', 3), ('etymology', 2), ('terminology', 1), ('definition', 4), ('wilhelm', 2), ('weitling', 2), ('example', 3), ('writer', 1), ('added', 1), ('theory', 10), ('without', 10), ('using', 1), ('exact', 1), ('term', 4), ('etymological', 1), ('origin', 1), ('ancient', 2), ('greek', 1), ('anarkhia', 1), ('meaning', 3), ('ruler', 3), ('composed', 2), ('prefix', 1), ('word', 1), ('arkhos', 1), ('leader', 2), ('suffix', 1), ('ism', 1), ('denotes', 1), ('ideological', 2), ('current', 6), ('favours', 2), ('appears', 1), ('english', 2), ('1642', 1), ('anarchisme', 1), ('1539', 1), ('early', 6), ('usages', 1), ('emphasised', 1), ('sense', 3), ('disorder', 1), ('factions', 2), ('within', 12), ('french', 6), ('revolution', 10), ('labelled', 1), ('opponents', 1), ('although', 7), ('accused', 1), ('shared', 2), ('many', 16), ('views', 3), ('later', 6), ('revolutionaries', 2), ('william', 6), ('godwin', 4), ('1756', 1), ('1836', 1), ('1808', 1), ('1871', 2), ('contribute', 1), ('doctrines', 2), ('next', 1), ('generation', 1), ('use', 7), ('describing', 1), ('beliefs', 2), ('philosopher', 3), ('call', 2), ('pierre-joseph', 5), ('proudhon', 6), ('1809', 1), ('1865', 1), ('marking', 1), ('birth', 2), ('mid-19th', 1), ('since', 4), ('1890s', 2), ('beginning', 2), ('france', 3), ('libertarianism', 4), ('often', 6), ('used', 5), ('synonym', 3), ('still', 5), ('common', 8), ('outside', 1), ('united', 4), ('hand', 4), ('refer', 2), ('individualistic', 1), ('free-market', 2), ('referring', 2), ('largely', 6), ('synonymous', 1), ('recently', 1), ('diluted', 1), ('wider', 3), ('adoption', 1), ('ideologically', 3), ('disparate', 2), ('groups', 14), ('including', 3), ('new', 7), ('left', 7), ('marxists', 4), ('associate', 1), ('authoritarian', 2), ('socialists', 3), ('vanguard', 1), ('party', 2), ('well', 4), ('extreme', 2), ('liber
  freq_list = docid_to_freq[2] # list of tuples
  lst_tf_idf = []
  doc_size = docid_to_freq[1]
  for freq in freq_list:
    term = freq[0]
    num = freq[1]
    df = df_dict[term]
    lst_tf_idf.append(((num/doc_size) * math.log(total_number_of_docs_N/df ,2))**2)
  return (docid_to_freq[0], our_sum(lst_tf_idf)**0.5)

In [7]:
doc_tf_vector_not_filtered = doc_text_pairs.map(lambda x: (x[1], word_count(x[0], x[1])))

doc_tf_vector = doc_tf_vector_not_filtered.filter(lambda x: len(x[1]) > 0)

doc_len = doc_tf_vector.map(lambda x: (x[0], our_sum([i[1][1] for i in x[1]]))) ## Di # 2 # key = dicId!!! value = doc len

doc_tf_unnormalized = doc_tf_vector.map(lambda x:(x[0], [(i[0],i[1][1]) for i in x[1]])) ## f_(i,j) # 3 # key = docId!!! value = tf vector

word_counts_not_filtered = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

word_counts = word_counts_not_filtered.filter(lambda x: len(x[1]) > 0)

postings = word_counts.groupByKey().mapValues(reduce_word_counts)

postings_filtered = postings.filter(lambda x: len(x[1])>10)

df_for_terms = postings_filtered.map(lambda x: (x[0], len(x[1])))

term_df = df_for_terms.collectAsMap()

doc_tf_unnormalized_with_doc_len = doc_tf_unnormalized.map(lambda x : (x[0],our_sum([i[1] for i in x[1]]),x[1]))

doc_vector_size = doc_tf_unnormalized_with_doc_len.map(lambda x : vector_size(x,term_df))

In [8]:
from functools import total_ordering

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  by_bucket = postings.map(lambda x : (token2bucket_id(x[0]), (x[0], x[1]))) # assuming a and b are lists then combining them
  by_bucket = by_bucket.groupByKey()
  return by_bucket.map(lambda x : InvertedIndex.write_a_posting_list(x))

posting_locs_list = partition_postings_and_write(postings_filtered).collect()

super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

In [9]:
# Create inverted index instance
body_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
body_index.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
body_index.df = term_df
body_index.doc_len = doc_len.collectAsMap()
body_index.doc_vector_size = doc_vector_size.collectAsMap()
body_index.N = total_number_of_docs_N
# write the global stats out
body_index.write_index('.', 'body_index_for_only_queries_docs')

Py4JJavaError: ignored